In [4]:
from fredapi import Fred
import pandas as pd

# Replace with your actual FRED API key
fred = Fred(api_key="b1ad22b04bdaee122b70f142df6af600")

# Dictionary of metal-related FRED series
metal_series = {
    "Gold Ores": "WPU102501",
    "Silver Ores": "WPU102502",
    "Copper Ores": "WPU102503",
    "Lead Ores": "WPU102504",
    "Zinc Ores": "WPU102505",
    "Nickel Ores": "WPU102506",
    "Aluminum Ores": "WPU102507",
    "Iron Ores": "WPU102508",
    "Molybdenum Ores": "WPU102509",
    "Tungsten Ores": "WPU102510",
    "Tin Ores": "WPU102511",
    "Cobalt Ores": "WPU102512",
    "Manganese Ores": "WPU102513",
    "Titanium Ores": "WPU102514",
    "Chromium Ores": "WPU102515",
    "Vanadium Ores": "WPU102516",
    "Lithium Ores": "WPU102517",
    "Magnesium Ores": "WPU102518",
    "Bismuth Ores": "WPU102519",
    "Antimony Ores": "WPU102520",
    "Cadmium Ores": "WPU102521",
    "Mercury Ores": "WPU102522",
    "Rare Earth Ores": "WPU102523",
    "Uranium Ores": "WPU102524",
    "Platinum Ores": "WPU102525",
    "Palladium Ores": "WPU102526",
    "Rhodium Ores": "WPU102527",
    "Osmium Ores": "WPU102528",
    "Iridium Ores": "WPU102529",
    "Ruthenium Ores": "WPU102530",
    "Import Price Index (End Use): Gem Diamonds": "IR42100",
    "Import Price Index (HS): Diamonds, Not Mounted or Set": "IP7102",
    "Export Price Index (HS): Diamonds, Not Mounted or Set": "ID7102"
}

# Helper function to calculate percentage change
def compute_change(series, periods):
    series = series.dropna()
    if len(series) < periods:
        return None
    start = series.iloc[-periods]
    end = series.iloc[-1]
    return round(((end - start) / start) * 100, 2)

# Retrieve and compute data
results = []

for name, code in metal_series.items():
    print(f"Fetching data for: {name}")
    try:
        series = fred.get_series(code, frequency='m')  # Monthly frequency
        row = {
            "Indicator": name,
            "Code": code,
            "1M Change %": compute_change(series, 1),
            "3M Change %": compute_change(series, 3),
            "6M Change %": compute_change(series, 6),
            "1Y Change %": compute_change(series, 12),
            "5Y Change %": compute_change(series, 60)
        }
        results.append(row)
    except Exception as e:
        print(f"Error fetching data for {name}: {e}")

# Save results to CSV
df = pd.DataFrame(results)
df.to_csv("metal_indicators.csv", index=False)
print("Data saved to metal_indicators.csv")
print(df)



Fetching data for: Gold Ores
Fetching data for: Silver Ores
Fetching data for: Copper Ores
Error fetching data for Copper Ores: Bad Request.  The series does not exist.
Fetching data for: Lead Ores
Fetching data for: Zinc Ores
Fetching data for: Nickel Ores
Error fetching data for Nickel Ores: Bad Request.  The series does not exist.
Fetching data for: Aluminum Ores
Error fetching data for Aluminum Ores: Bad Request.  The series does not exist.
Fetching data for: Iron Ores
Error fetching data for Iron Ores: Bad Request.  The series does not exist.
Fetching data for: Molybdenum Ores
Error fetching data for Molybdenum Ores: Bad Request.  The series does not exist.
Fetching data for: Tungsten Ores
Error fetching data for Tungsten Ores: Bad Request.  The series does not exist.
Fetching data for: Tin Ores
Error fetching data for Tin Ores: Bad Request.  The series does not exist.
Fetching data for: Cobalt Ores
Error fetching data for Cobalt Ores: Bad Request.  The series does not exist.
Fetc

In [2]:
pip install fredapi

In [ ]:
import pandas as pd
import time
import requests
from datetime import datetime, timedelta

API_KEY = 'YOUR_ALPHA_VANTAGE_KEY'  # Replace this with your actual key

# Load scraped mutual funds (from Yahoo Finance)
df = pd.read_csv("yahoo_mutualfund_gainers.csv")  # Or use the df from earlier scraping

# Function to fetch 2Y and 3Y returns from Alpha Vantage
def fetch_alpha_returns(ticker, retries=3):
    url = 'https://www.alphavantage.co/query'
    params = {
        'function': 'TIME_SERIES_DAILY',
        'symbol': ticker,
        'apikey': API_KEY,
        'outputsize': 'full'
    }

    for attempt in range(retries):
        try:
            r = requests.get(url, params=params, timeout=30)
            data = r.json()

            if 'Time Series (Daily)' not in data:
                print(f"❌ {ticker}: No data returned.")
                return None, None

            ts = data['Time Series (Daily)']
            df_price = pd.DataFrame.from_dict(ts, orient='index')
            df_price.index = pd.to_datetime(df_price.index)
            df_price = df_price.rename(columns={'4. close': 'close'})
            df_price['close'] = df_price['close'].astype(float)
            df_price = df_price.sort_index()

            today = df_price.index[-1]
            price_now = df_price.loc[today]['close']

            # Calculate 2Y and 3Y ago
            price_2y = df_price[df_price.index <= today - timedelta(days=730)].iloc[-1]['close']
            ret_2y = ((price_now - price_2y) / price_2y) * 100

            price_3y = df_price[df_price.index <= today - timedelta(days=1095)].iloc[-1]['close']
            ret_3y = ((price_now - price_3y) / price_3y) * 100

            return ret_2y, ret_3y

        except Exception as e:
            print(f"⚠️ {ticker}: Attempt {attempt+1} failed – {e}")
            time.sleep(5)
    return None, None

# Step: Fetch returns for each mutual fund
returns_2y = []
returns_3y = []

for i, row in df.iterrows():
    ticker = row['Ticker']
    ret_2y, ret_3y = fetch_alpha_returns(ticker)
    returns_2y.append(ret_2y)
    returns_3y.append(ret_3y)

    print(f"✅ {ticker}: 2Y = {ret_2y if ret_2y is not None else 'NA'}%, 3Y = {ret_3y if ret_3y is not None else 'NA'}%")
    time.sleep(15)  # Alpha Vantage free tier: 5 requests/minute

# Add to dataframe
df['2Y Return (%)'] = returns_2y
df['3Y Return (%)'] = returns_3y

# Save updated file
df.to_csv("mutual_fund_with_alpha_returns.csv", index=False)
print("\n📁 Results saved to mutual_fund_with_alpha_returns.csv")

❌ 0P0001T9Z1: No data returned.
✅ 0P0001T9Z1: 2Y = NA%, 3Y = NA%
✅ FTPAX: 2Y = -4.37452033768226%, 3Y = -7.360594795539022%
❌ 0P000150HP: No data returned.
✅ 0P000150HP: 2Y = NA%, 3Y = NA%
❌ 0P0001ALLY: No data returned.
✅ 0P0001ALLY: 2Y = NA%, 3Y = NA%
❌ 0P0001A9KH: No data returned.
✅ 0P0001A9KH: 2Y = NA%, 3Y = NA%
❌ 0P0001J4GT: No data returned.
✅ 0P0001J4GT: 2Y = NA%, 3Y = NA%
❌ 0P0001QQI1: No data returned.
✅ 0P0001QQI1: 2Y = NA%, 3Y = NA%
❌ 0P0001OU0G: No data returned.
✅ 0P0001OU0G: 2Y = NA%, 3Y = NA%
❌ 0P0001KYOC: No data returned.
✅ 0P0001KYOC: 2Y = NA%, 3Y = NA%
❌ 0P0001NGQD: No data returned.
✅ 0P0001NGQD: 2Y = NA%, 3Y = NA%
❌ 0P000162E2: No data returned.
✅ 0P000162E2: 2Y = NA%, 3Y = NA%
❌ 0P000162E4: No data returned.
✅ 0P000162E4: 2Y = NA%, 3Y = NA%
❌ 0P000162E1: No data returned.
✅ 0P000162E1: 2Y = NA%, 3Y = NA%
❌ 0P000162E3: No data returned.
✅ 0P000162E3: 2Y = NA%, 3Y = NA%
❌ 0P0000A2W1: No data returned.
✅ 0P0000A2W1: 2Y = NA%, 3Y = NA%
❌ 0P0000A2VZ: No data returned.

In [ ]:
import requests

url = "https://query1.finance.yahoo.com/v1/finance/screener/predefined/saved"
params = {"scrIds": "mutualfund_day_gainers", "count": 100, "start": 0}

r = requests.get(url, params=params)
data = r.json()

for item in data["finance"]["result"][0]["quotes"]:
    print(f"{item['shortName']} ({item['symbol']})")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)